In [1]:
! git clone https://bitbucket.org/spekpy/spekpy_release.git

Cloning into 'spekpy_release'...
remote: Enumerating objects: 973, done.
remote: Counting objects: 100% (973/973), done.
remote: Compressing objects: 100% (306/306), done.
remote: Total 973 (delta 806), reused 683 (delta 649), pack-reused 0
Receiving objects: 100% (973/973), 90.22 MiB | 11.54 MiB/s, done.
Resolving deltas: 100% (806/806), done.


In [2]:
! pip install spekpy_release/.

Processing ./spekpy_release
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 638.1 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.3/30.3 MB 17.7 MB/s eta 0:00:0000:0100:01
  Created wheel for spekpy: filename=spekpy-2.0.11.1-py3-none-any.whl size=82882066 sha256=ac0b81882c84ed92388fc896e4cad88032e6a17079644beb3a372927d4960111
  Stored in directory: /private/var/folders/n_/_z_q62kx279gp1tfsgqh78s40000gn/T/pip-ephem-wheel-cache-0934_jk2/wheels/41/c8/42/453fe99eea5b37d60064e142295ef33cf8caae64fc528c446d
Successfully built spekpy


In [3]:
import numpy as np
from matplotlib import pyplot as plt
import spekpy as sp
import pandas as pd
#print("\n**Script to generate a filtered spectrum and print summary statistics to screen **\n")

In [4]:
# Define x-ray tube parameters

# CHANGE THESE PARAMETERS
kvp = [40] # Tube potential in kV (default: 100)
z = 100 # Distance from source
mas = 35  # Tube current


# DON'T CHANGE THESE
th = 20 # Anode angle in degrees (default: 12)
dk = 0.5 # Spectrum bin width in keV (default: 0.5)
physics = "default" # Legacy physics mode rather than default (default: "default")
mu_data_source = "nist" # Penelope mu/mu_en data rather than NIST/XCOM (default: "nist")
x = 0 # Point-of-interest displaced 5 cm towards cathode in anode-cathode direction (default: 0)
y = 0 # Point-of-interest displaced -5 cm in orthogonal direction (right-hand-rule is applicable) (default: 0)
brem = True # Whether the bremsstrahlung portion of spectrum is retained (default: True)
char = True # Whether the characteristic portion of spectrum is retained (default: True)
obli = True # Whether path-length through extrinsic filtration varies with x, y (default: True)


In [5]:
# Generate spectrums for each energy in kvp list

# use Air Kerma for dose rate

for e in kvp:
    print('kvp = ',str(e))
    s = sp.Spek(kvp=e, th=th, dk=dk, physics=physics, mu_data_source=mu_data_source,
      x=x, y=y, z=z, mas=mas, brem=brem, char=char, obli=obli)

    # Uncomment the line below and you can filter the spectrum with copper
    # s.filter('C',0.125) # carbon, mm
    # s.filter('Cu',0.36)

    # Print full summary of spectrum metrics
    s.summarize(mode='full')


    fluence = s.get_flu()  # photons/ (s * cm^2)
    conver = 1.6022E-16  # J per keV
    spectrum = s.get_spectrum()
    energies = np.copy(spectrum[0])  # Energies only
    spectra = np.copy(spectrum[1])  # Weights only
    spectra = spectra / np.linalg.norm(spectra, ord=1)  # Normalize to sum to 1
    spectra = spectra * fluence  # Now the spectra weights sum to the total fluence

    energies = energies * conver  # Convert energies to J

    total_energy = np.multiply(energies, spectra)
    total_power = np.sum(total_energy)
    print(rf'X-ray power: {total_power * 1E3:0.5f} mW/cm^2')
    print()

kvp =  40
Inputs
------
Tube Voltage: 40.0 [kVp]; Anode Angle: 20.0 [degrees]; Energy Bin: 0.5 [keV]; Bin shift fraction: None []; Physics Mode: spekpy-v1 [str]; Mu Data Source: nist [str]; Target: W [str]; 
x: 0.0 [cm]; y: 0.0 [cm]; z: 100.0 [cm]; Tube Load: 35.0 [mAs]; Bremsstrahlung Emission: True [bool]; Characteristic Emission: True [bool]; Oblique: True [bool]; Ref. air Kerma: None [uGy]; Ref. fluence: None [Photons cm^-2]; 
Filtration: [] [mm]; 
Outputs
-------
Fluence: 17420606492.7876 [Photons cm^-2];  Air Kerma: 406240.2050 [uGy]; Mean Energy: 11.2923 [keV]; 
First HVL Al: 0.0049 [mm Al];  Second HVL Al: 0.0192 [mm Al];  Homogeneity Coefficient Al: 0.2561 [-]; Effective Energy Al: 3.4906 [keV]; 
First HVL Cu: 0.0005 [mm Cu];  Second HVL Cu: 0.0038 [mm Cu];  Homogeneity Coefficient Cu: 0.1413 [-]; Effective Energy Cu: 9.2500 [keV]; 

X-ray power: 0.03152 mW/cm^2

